In [23]:
load('Imports/Utility.sage', 'Imports/Point.sage', 'Imports/Line.sage', 'Imports/Tritangent_plane.sage', 'Imports/Group.sage', 'Imports/Cubic.sage')
import multiprocessing as mp
import pickle

In [24]:
P.<x,y,z,t,b,c,d,e,f,l,m> = PolynomialRing(QQ)

In [25]:
cubic_new = e*f*(2*x^2*y-2*x*y^2+x*z^2-x*z*t-y*t^2+y*z*t)+b*c*(x-t)*(x*z+y*t)+c*c*(z+t)*(y*t-x*z)+d*c*(y-z)*(x*z+y*t)+(e+f)*c*(x-y)*(y*t-x*z)

In [26]:
sing_cubics = (-1) * (-c + f) * (-c + e) * c * (c + f) * (c + e) * (-e + f)^2 * (-c*d + c*f + e*f) * (-c*d + c*e + e*f) * (-c^2 - c*d + e*f) * (b*c - c*f + e*f) * (b*c - c*e + e*f) * (b*c - c*d + 2*e*f) * (b*c - c^2 + e*f) * (b*c^2 + c^2*d + b*c*f - 2*c^2*f - c*d*f + 2*e*f^2) * (b*c^2 + c^2*d + b*c*e - 2*c^2*e - c*d*e + 2*e^2*f) * (-b*c^3 - 2*b*c^2*d + c^3*d + b*c^2*e + c^2*d*e + b*c^2*f + c^2*d*f + 3*b*c*e*f - 4*c^2*e*f - 3*c*d*e*f + 4*e^2*f^2)

In [27]:
line = Line([y, z])

In [28]:
eqn_SE1 = P(cubic_new.subs({b:-(c*c+e*f)/c }) )

In [29]:
sing_cubics_SE1 = P(sing_cubics.subs({b:-(c*c+e*f)/c }))

In [30]:
SE1 = Cubic(eqn_SE1, line, sing_cubics_SE1.factor())

In [31]:
SE1.eckardt_points

[(0, 0, 0, 1)]

In [32]:
%time adm_SE1 = SE1.find_admissible_projectivities()

CPU times: user 1.35 s, sys: 210 ms, total: 1.56 s
Wall time: 10.2 s


In [33]:
len(adm_SE1)

576

In [34]:
%time simm_SE1 = SE1.find_simmetries(adm_SE1)

CPU times: user 1.15 s, sys: 161 ms, total: 1.31 s
Wall time: 5.04 s


In [35]:
len(simm_SE1)

2

In [36]:
G_SE1 = Group(simm_SE1)

In [37]:
G_SE1.is_group()

True

In [38]:
G_SE1.get_order()

2

In [39]:
G_SE1.is_abelian()

True

In [40]:
G_SE1.get_divisors_of_order()

[1, 2]

In [41]:
G_SE1.get_order_of_elements()

[1, 2]

In [42]:
G_SE1.apply_to(SE1.eckardt_points[0])

[(0, 0, 0, 1), (0, 0, 0, 1)]

In [43]:
def find_conditions_for_subfamilies(cubic, sing_cubics, projectivities, simmetries):
    mon = ((x+y+z+t)^3).monomials()
    conditions = []
    sing_cubics_factored = sing_cubics.factor()
    for M in [proj for proj in projectivities if proj not in simmetries]:
        sost = change_coord(M)
        new_cubic = remove_sing_factors(cubic.eqn.subs(sost), sing_cubics_factored)    
        minor = matrix([[new_cubic.coefficient(mn) for mn in mon], [cubic.eqn.coefficient(mn) for mn in mon]]).minors(2)
        minor = [remove_sing_factors(el, sing_cubics_factored) for el in minor if el !=0]
        prim_deco = P.ideal(minor).radical().primary_decomposition()
        for ideale in prim_deco:
            gener = tuple([remove_sing_factors(gen, sing_cubics_factored) for gen in ideale.gens()])
            if 1 not in gener:
                conditions.append(gener)    
    return list(set(conditions))

In [44]:
def apply_perm(perm, keys, base_keys):
    new_keys_index = [perm.index(label) for label in base_keys]
    new_keys_index, keys = (list(t) for t in zip(*sorted(zip(new_keys_index, keys))))
    return keys